In [1]:
import pandas as pd

### Checking Single Datasets for Duplicate Entries

In [4]:
pneumonia = pd.read_excel("Viral Pneumonia.matadata.xlsx").assign(label="non-covid-19 pneumonia", view="AP")
pneumonia.shape

(1345, 6)

In [ ]:
pneumonia_dropped = pneumonia.drop_duplicates()
pneumonia_dropped.shape

In [5]:
covid19 = pd.read_excel("COVID-19.metadata.xlsx").assign(label="covid-19 pneumonia",view="AP")
covid19.shape

(219, 6)

In [6]:
covid19_dropped = covid19.drop_duplicates()
covid19.shape

(219, 6)

In [7]:
normal = pd.read_excel("NORMAL.metadata.xlsx").assign(label="non-infection", view="AP")
normal.shape

(1341, 6)

In [8]:
normal_dropped = normal.drop_duplicates()
normal_dropped.shape

(1341, 6)

In [12]:
chest = pd.read_csv("covid_chestxray_metadata.csv")
chest.shape

(930, 29)

In [13]:
chest_dropped = chest.drop_duplicates()
chest_dropped.shape

(930, 29)

In [14]:
figure1 = pd.read_csv("figure1_covid_chestxray_metadata.csv", encoding='iso-8859-1')
figure1.shape

(55, 12)

In [15]:
figure1_dropped = figure1.drop_duplicates()
figure1_dropped.shape

(55, 12)

In [17]:
actual_med = pd.read_csv("actualmed_covid_chestxray_metadata.csv")
actual_med.shape

(238, 13)

In [18]:
actual_med_dropped = actual_med.drop_duplicates()
actual_med_dropped.shape

(238, 13)

### Checking Duplicate Values between the datasets

In [19]:
counta = 0
countb = 0
countc = 0
for i in pneumonia.index:
    link = pneumonia['URL'][i]
    for a in covid19.index:
        if(covid19['URL'][a] == link):
             counta += 1

for i in pneumonia.index:
    link = pneumonia['URL'][i]    
    for b in normal.index:
        if(normal['URL'][b] == link):
             countb += 1

for i in pneumonia.index:
    link = pneumonia['URL'][i]
    for c in chest.index:
        if(chest['url'][c] == link):
             countc += 1

print (counta)
print (countb)
print (countc)
   
        

0
1803645
0


## The above result shows the similar url from pneumonia compared to every other dataset, normal shows this many figures because every links are similar in both dataset.

## The below result shows similar values in dataset covid19 and chest xray.

In [20]:
duplicate_chest = []
counta = 0
countb = 0
for i in covid19.index:
    link = covid19['URL'][i]
    for a in normal.index:
        if(normal['URL'][a] == link):
             counta += 1
print(counta)

for i in covid19.index:
    link = covid19['URL'][i]
    for b in chest.index:
        if(chest['url'][b] == link):
            countb += 1
            duplicate_chest.append(b)


print(countb)


0
487


## 134 Duplicate values were removed from chest Xray dataset

In [21]:
chestx = chest.drop(chest.index[duplicate_chest])
chestx.shape


(796, 29)

In [22]:
chest.shape

(930, 29)

## No similar values were found between normal and chest xray dataset

In [23]:
counta = 0
for i in normal.index:
    link = normal['URL'][i]
    
    for a in chest.index:
        if(chest['url'][a] == link):
            counta += 1
            
print(counta)

0


## Merging the Datasets

In [24]:
for i in normal.index:
    image = normal['FILE NAME'][i] 
    ext =  normal['FORMAT'][i]    
    root =  "Radiography"   
    normal['FILE NAME'][i] = root+'/'+image+'.'+ext
for i in covid19.index:
    image = covid19['FILE NAME'][i] 
    ext =  covid19['FORMAT'][i]
    root =  "Radiography"   
    covid19['FILE NAME'][i] = root+'/'+image+'.'+ext
for i in pneumonia.index:
    image = pneumonia['FILE NAME'][i] 
    ext =  pneumonia['FORMAT'][i]  
    root =  "Radiography"   
    pneumonia['FILE NAME'][i] = root+'/'+image+'.'+ext

radiography = pd.concat([normal, covid19, pneumonia])

In [25]:
radiography

,FILE NAME,FORMAT,SIZE,URL,label,view
0,Radiography/NORMAL-1.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-infection,AP
1,Radiography/NORMAL-2.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-infection,AP
2,Radiography/NORMAL-3.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-infection,AP
3,Radiography/NORMAL-4.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-infection,AP
4,Radiography/NORMAL-5.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-infection,AP
...,...,...,...,...,...,...
1340,Radiography/Viral Pneumonia-1341.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-covid-19 pneumonia,AP
1341,Radiography/Viral Pneumonia-1342.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-covid-19 pneumonia,AP
1342,Radiography/Viral Pneumonia-1343.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-covid-19 pneumonia,AP
1343,Radiography/Viral Pneumonia-1344.PNG,PNG,1024*1024,https://www.kaggle.com/paultimothymooney/chest...,non-covid-19 pneumonia,AP


In [26]:
df = radiography[["FILE NAME", "label", "view"]]
merged =df.rename(columns={"FILE NAME":"filename"})


In [27]:
def convert_finding_to_label(finding):
    if "COVID-19" in finding:
        return "covid-19 pneumonia"
    return "non-covid-19 pneumonia"

chestx= chestx.assign(label=chest.finding.apply(convert_finding_to_label), dataset="covid_chestxray")

### Deleting entries for the images which does not exist in dataset

In [28]:
import os
def check_file(filename):
   
    if os.path.isfile(filename):
        return 1 
    else: 
        return 0
    
arr = []
for i in chestx.index:
    image = chestx['filename'][i] 
    root =  "covid_chestxray"   
    chestx['filename'][i] = root+'/'+image
    val = check_file(chestx['filename'][i])
    if val == 0:
        arr.append(i)

C:\Users\Manoj\anaconda3\envs\tf-keras-gpu-test\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [29]:
print(chestx.shape)

for i in arr:
    chestx = chestx.drop(index = i)    

print(chestx.shape)


(796, 31)
(777, 31)


In [30]:
merged = pd.concat([merged, chestx[["filename","label","view"]]])

In [31]:
merged

,filename,label,view
0,Radiography/NORMAL-1.PNG,non-infection,AP
1,Radiography/NORMAL-2.PNG,non-infection,AP
2,Radiography/NORMAL-3.PNG,non-infection,AP
3,Radiography/NORMAL-4.PNG,non-infection,AP
4,Radiography/NORMAL-5.PNG,non-infection,AP
...,...,...,...
925,covid_chestxray/e0e3a6526a3fecadfca2be13242798...,non-covid-19 pneumonia,L
926,covid_chestxray/2264f643b18b1010ec10a850f17550...,non-covid-19 pneumonia,PA
927,covid_chestxray/800f798a58d0cbcc72eb234f192461...,non-covid-19 pneumonia,L
928,covid_chestxray/55f5189d2c23688ac8dc1d58eb65cf...,non-covid-19 pneumonia,PA


In [32]:
def convert_figure1_finding(finding):
    if finding == "No finding":
        return "non-infection"
    elif finding == "COVID-19":
        return "covid-19 pneumonia"
    elif finding == "Pneumonia":
        return "non-covid-19 pneumonia"
    
import os
def find_extension(filename):
    path = os.path.join("figure1", filename) 
    png = path+'.png'
    jpg = path+'.jpg'
  
    if os.path.isfile(png):
        return "figure1/"+filename+".png" 
    elif os.path.isfile(jpg):
        return "figure1/"+filename+".jpg"
    
figure1 = figure1.assign(url='', dataset="figure1",
                         filename =figure1.patientid.apply(find_extension),
                         label=figure1.finding.apply(convert_figure1_finding)).dropna(subset=["label"])



In [33]:
merged = pd.concat([merged, figure1[["filename","label","view"]]])

In [34]:
merged.shape

(3722, 3)

In [35]:
actual_med = actual_med.assign(dataset="actualmed",
                         filename ='',
                         label=actual_med.finding.apply(convert_figure1_finding)).dropna(subset=["label"])


In [36]:
for i in actual_med.index:
    image = actual_med['imagename'][i] 
    root =  "actual_med"   
    actual_med['filename'][i] = root+'/'+image

C:\Users\Manoj\anaconda3\envs\tf-keras-gpu-test\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [37]:
actual_med.shape

(185, 16)

In [38]:
merged = pd.concat([merged, actual_med[["filename","label","view"]]])

In [39]:
merged.shape

(3907, 3)

In [40]:
merged.isnull().sum()

filename     0
label        0
view        32
dtype: int64

In [41]:
merged.columns

Index(['filename', 'label', 'view'], dtype='object')

## Saving the merged files

In [42]:
merged.to_csv("merged_data_final.csv")